<a href="https://colab.research.google.com/github/kanithasa/pandas/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [3]:
file  = open('AA_wiki_00.txt', 'r', encoding = "utf8")
lines=[]
for i in file:
    lines.append(i)
lines=lines[9:]
data=""
for i in lines:
    data = ' '.join(lines)
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')
data = data.split()
data = ' '.join(data)

In [4]:
data[:1000]

'கட்டிடக்கலை என்பது கட்டிடங்கள் மற்றும் அதன் உடல் கட்டமைப்புகளை வடிவமைத்தல், செயல்முறைத் திட்டமிடல், மற்றும் கட்டிடங்கள் கட்டுவதை உள்ளடக்கியதாகும். கட்டடக்கலை படைப்புகள், கட்டிடங்கள் பொருள் வடிவம், பெரும்பாலும் கலாச்சார சின்னங்களாக மற்றும் கலை படைப்புகளாக காணப்படுகின்றது. வரலாற்று நாகரிகங்கள் பெரும்பாலும் அவர்களின் கட்டிடகலை சாதனைகளின் மூலம் அடையாளம் காணப்படுகின்றன. ஒரு விரிவான வரைவிலக்கணம், பெருமட்டத்தில், நகரத் திட்டமிடல், நகர்ப்புற வடிவமைப்பு மற்றும் நிலத்தோற்றம் முதலியவற்றையும், நுண்மட்டத்தில், தளபாடங்கள், உற்பத்திப்பொருள் முதலியவற்றை உள்ளடக்கிய, முழு உருவாக்கச் சூழலின் வடிவமைப்பைக் கட்டிடக்கலைக்குள் அடக்கும். மேற்படி விடயத்தில், தற்போது கிடைக்கும் மிகப் பழைய ஆக்கம், கி.பி. முதலாம் நூற்றாண்டைச் சேர்ந்த உரோமானியக் கட்டடக் கலைஞரான விட்ருவியஸ் என்பாரது "கட்டிடக்கலை தொடர்பில்", என்ற நூலாகும். இவரது கூற்றுப்படி, நல்ல கட்டிடம், அழகு, உறுதி, பயன்பாடு ஆகியவற்றைக் கொண்டிருக்கவேண்டும். மேற்படி மூன்றும், ஒன்றின்மீதொன்று அளவுமீறி ஆதிக்கம் செலுத்தாமல், தங்களிடையே சமனிலையையும், ஒருங்கினைப்பையும்

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
pickle.dump(tokenizer, open('token.pkl', 'wb'))
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]
vocab_size = len(tokenizer.word_index)+1
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [6]:
len(sequence)

41427

In [7]:
sequence = np.array(sequence)

In [8]:
sequence

array([[   80,    11,   120,     2],
       [   11,   120,     2,    46],
       [  120,     2,    46,   534],
       ...,
       [17852,   425, 17853, 17854],
       [  425, 17853, 17854, 17855],
       [17853, 17854, 17855,     4]])

In [9]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

X=np.array(X)
y=np.array(y)
y = to_categorical(y, num_classes=vocab_size)

In [10]:
X

array([[   80,    11,   120],
       [   11,   120,     2],
       [  120,     2,    46],
       ...,
       [17852,   425, 17853],
       [  425, 17853, 17854],
       [17853, 17854, 17855]])

In [11]:
len(X),len(y)

(41427, 41427)

In [12]:
y

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

max_seq_length = max(len(seq) for seq in sequence)


# Initialize model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=3))  # Input length is now 3
model.add(LSTM(units=256))
model.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X,y, epochs=10, batch_size=32)

Epoch 1/10
1295/1295 [==============================] - 159s 120ms/step - loss: 9.4282 - accuracy: 0.0175
Epoch 2/10
1295/1295 [==============================] - 150s 116ms/step - loss: 8.6513 - accuracy: 0.0305
Epoch 3/10
 371/1295 [=======>......................] - ETA: 1:48 - loss: 7.7106 - accuracy: 0.0323

In [ ]:
model.summary()

In [ ]:
model.save('nlp1.h5')

In [ ]:
def predict_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds=np.argmax(model.predict(sequence))
    predicted_word = ""
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    print(predicted_word)
    return predicted_word

In [ ]:
predict_word(model, tokenizer, 'கட்டிடக்கலையின் வரலாற்றில் பாரிய')

In [ ]:
model.save('next_words.h5')

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

In [ ]:
model = load_model('next_words.h5')
# tokenizer = pickle.load(open('token.pkl', 'rb'))

In [ ]:
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import to_categorical
import numpy as np

for i in range(1,8):

    # Load the model
    model = load_model('next_words.h5')


    with open(f'AA_wiki_0{i}.txt', 'r', encoding='utf8') as file:
        lines =[h for h in file.readlines()[9:] if not h[0].startswith("<")]   # Skip the first 9 lines
        data=""
        for i in lines:
            data = ' '.join(lines)
        data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')
        data = data.split()
        data = ' '.join(data)
        print(data)
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts([data])
        # pickle.dump(tokenizer, open('token.pkl', 'wb'))
        sequence_data = tokenizer.texts_to_sequences([data])[0]

    vocab_size = len(tokenizer.word_index) + 1
    print(vocab_size)

    sequence=[]
    for i in range(3, len(sequence_data)):
        words = sequence_data[i-3:i+1]
        sequence.append(words)
    sequence=np.array(sequence)
    X, y = [], []
    for i in sequence:
        X.append(i[0:3])
        y.append(i[3])

    X = np.array(X)
    y = np.array(y)

    # Convert y to categorical
    y = to_categorical(y, num_classes=vocab_size)

    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

    # Train the model
    checkpoint = ModelCheckpoint("next_words.h5", monitor="loss", verbose=1, save_best_only=True)
    model.fit(X, y, epochs=10, batch_size=64, callbacks=[checkpoint])

